# Calculating Shiller's CAPE measure

In [7]:
import numpy as np
import pandas as pd

In [49]:
shiller = pd.read_excel('http://www.econ.yale.edu/~shiller/data/ie_data.xls',
                            skiprows=range(7), skipfooter=1, usecols="A:E")

shiller.head()

,Date,P,D,E,CPI
0,1871.01,4.44,0.26,0.4,12.464061
1,1871.02,4.50,0.26,0.4,12.844641
2,1871.03,4.61,0.26,0.4,13.034972
3,1871.04,4.74,0.26,0.4,12.559226
4,1871.05,4.86,0.26,0.4,12.273812


In [50]:
shiller.tail(3)

,Date,P,D,E,CPI
1775,2018.12,2567.31,53.75,NaN,251.23300
1776,2019.01,2602.55,NaN,NaN,250.83050
1777,2019.02,2707.88,NaN,NaN,250.62925


In [53]:
shiller.sample(5)

,Date,P,D,E,CPI
1341,1982.10,132.70,6.85667,13.2533,98.2
669,1926.10,13.02,0.67500,1.2420,17.6
757,1934.02,11.32,0.44170,0.4483,13.3
1143,1966.04,91.60,2.79667,5.3800,32.3
562,1917.11,7.04,0.67920,1.3010,13.5


## Convert dates

The dates are not currently saved in a date format; pandas just thinks they floating-point numbers.

In [54]:
shiller.dtypes

Date    float64
P       float64
D       float64
E       float64
CPI     float64
dtype: object


Let's right a function that takes one of these numbers and converts it to a `pd.datetime` value.

In [62]:
def dtparse(dt):
    yr = int(dt)
    mo = int((dt - yr)*100) + 1
    return pd.datetime(yr, mo, 1)

In [25]:
# why do we need to add 1 to the month?
1871.01 - 1871

0.009999999999990905

In [21]:
# a different approach
def dtparse(dt):
    yr, mo = map(int, str(dt).split('.'))
    return pd.datetime(yr, mo, 1)

In [22]:
dtparse(1950.01)

datetime.datetime(1950, 1, 1, 0, 0)

In [23]:
dtparse(1950.12)

datetime.datetime(1950, 12, 1, 0, 0)

In [26]:
shiller['Date'].apply(dtparse)

NameError: name 'shiller' is not defined

In [59]:
shiller['Date'] = shiller['Date'].apply(dtparse)

In [60]:
shiller.head()

,Date,P,D,E,CPI
0,1871-01-01,4.44,0.26,0.4,12.464061
1,1871-02-01,4.50,0.26,0.4,12.844641
2,1871-03-01,4.61,0.26,0.4,13.034972
3,1871-04-01,4.74,0.26,0.4,12.559226
4,1871-05-01,4.86,0.26,0.4,12.273812


In [61]:
shiller.dtypes

Date    datetime64[ns]
P              float64
D              float64
E              float64
CPI            float64
dtype: object

In [ ]:
shiller = shiller.set_index('Date')
shiller.head()

In [ ]:
shiller.loc['1995-01-01']

When the index is a `datetime`, we can refer to ranges of dates in a meaningful way.

In [ ]:
shiller.loc['1995']

In [ ]:
shiller.loc['1995-02':'1995-04']

In [ ]:
shiller.iloc[-3:]

To convert all the data to "real" terms, we need to adjust for inflation. We do this by calculating how much the CPI has changed from each year until the last year in the sample.

In [ ]:
shiller['CPI'].iloc[-1]

In [ ]:
shiller['CPI'].iloc[-1] / shiller['CPI'].loc['2000']

In [ ]:
shiller['CPI_factor'] = shiller['CPI'].iloc[-1] / shiller['CPI']

In [ ]:
shiller.loc['1965']

In [ ]:
shiller['p_real'] = shiller['P'] * shiller['CPI_factor']
shiller['d_real'] = shiller['D'] * shiller['CPI_factor']
shiller['e_real'] = shiller['E'] * shiller['CPI_factor']

In [ ]:
shiller.head()

In [ ]:
shiller = shiller.drop(columns=['P', 'D', 'E', 'CPI', 'CPI_factor'])

In [ ]:
shiller.tail(15)

Let's calculate the *cyclically-adjusted price earnings ratio (CAPE)*. This is the price at a point in time divided by the average earnings over the previous ten years.

In [ ]:
# cyclically adjusted price earnings ratio
r = shiller.rolling(window=120, min_periods=100)
r['e_real'].mean()

The value of CAPE is the current price value divided by the rolling average *lagged one period*. We do this by calling `.shift` to shift the time series one period.

In [ ]:
shiller['CAPE'] = shiller['p_real'] / r['e_real'].mean().shift()

shiller.tail()

Then, we'll add a recession indicator variable (1 or 0) by accessing data from the St. Louis Federal Reserve data service, [FRED](https://fred.stlouisfed.org/). We can do this with the `pandas_datareader` package, a separate module designed to work with pandas and provide access to numerous online data sources.

The particular series we want is at https://fred.stlouisfed.org/series/USREC.

In [ ]:
import pandas_datareader as pdr

nber = pdr.get_data_fred('USREC', '1870')
nber.head()

The `shiller` DataFrame and the `nber` Series both have the same index (`datetime` values). We can *join* the two together on that index.

In [ ]:
shiller = shiller.join(nber).dropna()

shiller.head()

In [ ]:
# keep only data starting in 1880
shiller = shiller.loc['1880':]

shiller.head()

## Plotting

(This is more advanced. Don't worry about learning these details yet.)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator

# create Series with means and current value repeated
avg = pd.Series(shiller['CAPE'].mean(), index=shiller.index)
avg47 = pd.Series(shiller.loc['1947':,'CAPE'].mean(), index=shiller.loc['1947':].index)
curentval = pd.Series(shiller['CAPE'].iloc[-1], index=shiller.index)

# main plot
ax = shiller['CAPE'].plot(figsize=(9,6), legend=True, x_compat=True, ylim=(0,45), c='b', lw=1)

# change year locations
yrs20 = YearLocator(20)
yrs5 = YearLocator(5)
ax.xaxis.set_major_locator(yrs20)
ax.xaxis.set_minor_locator(yrs5)

# add bells & whistles
curentval.plot(ax=ax, label='Current level', legend=True, c='k', ls=':', lw=1)
avg.plot(ax=ax, label='Avg', legend=True, c='r', ls='--', lw=1)
avg47.plot(ax=ax, label='Avg, post-war', legend=True, c='g', ls='--', lw=1)
plt.legend(loc=0, fontsize='medium')
ax.fill_between(shiller['CAPE'].index, 0, 45, where=shiller['USREC']==1, 
                facecolor='lightgrey', edgecolor='lightgrey', alpha=0.5)

## Correlations

Let's calculate future 2- and 5-year returns. We do this by dividing the price in 2 or 5 years by the price "today".

In [ ]:
shiller['ret2yr'] = shiller['p_real'].shift(-2*12) / shiller['p_real'] - 1
shiller['ret5yr'] = shiller['p_real'].shift(-5*12) / shiller['p_real'] - 1

In [ ]:
shiller.head()

In [ ]:
shiller.tail()

In [ ]:
shiller[['CAPE', 'ret2yr', 'ret5yr']].corr()